In [ ]:
!pip install ultralytics "numpy<2"


In [1]:
import os
from pathlib import Path
import torch
import pandas as pd
import matplotlib.pyplot as plt
from ultralytics import YOLO
import cv2
import numpy as np
import seaborn as sns


In [2]:
# Point to your segmentation data.yaml
DATA_YAML_PATH = "/kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/data.yaml"  # change this

# Where to save results
OUTPUT_DIR = Path("/kaggle/working/yolov8s_seg_results")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

print(DATA_YAML_PATH)
print(OUTPUT_DIR)


/kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/data.yaml
/kaggle/working/yolov8s_seg_results


In [3]:
# Load COCO-pretrained YOLOv8s segmentation model
model = YOLO("yolov8s-seg.pt")

# Train
results = model.train(
    data=DATA_YAML_PATH,
    epochs=50,
    imgsz=640,
    batch=8,
    project=str(OUTPUT_DIR),
    name="yolov8s-seg",
    exist_ok=True
)


Ultralytics 8.3.235 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov8s-seg, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mas

In [4]:
best_model_path = OUTPUT_DIR / "yolov8s-seg" / "weights" / "best.pt"
print(best_model_path)


/kaggle/working/yolov8s_seg_results/yolov8s-seg/weights/best.pt


In [6]:
# Define image_files first
val_dir = Path(DATA_YAML_PATH).parent / "valid" / "images"
image_files = list(val_dir.glob("*.jpg"))[:6]

# Now loop through them
for i, img_path in enumerate(image_files):
    r = model.predict(str(img_path), conf=0.5)[0]
    annotated = r.plot()
    
    # Save instead of show
    save_path = OUTPUT_DIR / f"inference_sample_{i}.jpg"
    cv2.imwrite(str(save_path), annotated)
    print(f"✓ Saved: {save_path}")



image 1/1 /kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/valid/images/akhand_b43_242_jpg.rf.c1acbe3699cfbc0f571819aa4ed3240a.jpg: 640x640 2 Car-Damages, 20.4ms
Speed: 3.2ms preprocess, 20.4ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)
✓ Saved: /kaggle/working/yolov8s_seg_results/inference_sample_0.jpg

image 1/1 /kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/valid/images/severe_371_JPEG_jpg.rf.d383f95d374cb4559a99e86dce606f02.jpg: 640x640 1 Car-Damage, 20.4ms
Speed: 2.5ms preprocess, 20.4ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
✓ Saved: /kaggle/working/yolov8s_seg_results/inference_sample_1.jpg

image 1/1 /kaggle/input/car-damage-detection1/Car-Damage detection.v1i.yolov8/valid/images/test_set_188_jpg.rf.518a3f6a609385e9706aba936e77e9e8.jpg: 640x640 2 Car-Damages, 20.3ms
Speed: 2.6ms preprocess, 20.3ms inference, 1.7ms postprocess per image at shape (1, 3, 640, 640)
✓ Saved: /kaggle/working/yolo

In [1]:
# ============================================================
# CELL 1: INSTALL DEPENDENCIES
# ============================================================
!pip -q install ultralytics albumentations --no-deps


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.6 MB/s eta 0:00:0000:0100:01


In [3]:
%%writefile /kaggle/working/data.yaml
train: /kaggle/input/car-damage-detection/Car-Damage detection.v11.yolov8/train/images
val: /kaggle/input/car-damage-detection/Car-Damage detection.v11.yolov8/valid/images
test: /kaggle/input/car-damage-detection/Car-Damage detection.v11.yolov8/test/images

nc: 1

names: ['Car-Damage']


Overwriting /kaggle/working/data.yaml


In [4]:
# ============================================================
# CELL 3: IMPORTS
# ============================================================
import os
import glob
import random
import shutil
import yaml
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import gc
import torch

from ultralytics import YOLO
import albumentations as A


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [5]:
# ============================================================
# CELL 4: IMPROVED SETTINGS
# ============================================================
BASE_DATASET = "/kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8"
NUM_CLASSES = 1
CLASS_NAMES = ["Car-Damage"]
IMGSZ = 640
DEVICE = 0  # Single GPU
LABELED_FRACTION = 0.3  # INCREASED from 0.2

# MixMatch-specific settings - IMPROVED
MIXMATCH_WARMUP_EPOCHS = 60  # INCREASED from 40
MIXMATCH_EPOCHS = 70  # INCREASED from 50
BATCH_SIZE = 6
MIXMATCH_CONF = 0.25  # DECREASED from 0.5
K_AUGMENTATIONS = 4  # INCREASED from 2
MIN_AREA = 500  # NEW: Minimum area filter for pseudo-labels

# Paths
MM_LABELED_PATH = "/kaggle/working/mm_labeled"
MM_UNLABELED_PATH = "/kaggle/working/mm_unlabeled"
MM_PSEUDO_PATH = "/kaggle/working/mm_pseudo"
MM_MERGED_PATH = "/kaggle/working/mm_merged"

# Create directories
for p in [MM_LABELED_PATH, MM_UNLABELED_PATH, MM_PSEUDO_PATH, MM_MERGED_PATH]:
    os.makedirs(f"{p}/images", exist_ok=True)
    os.makedirs(f"{p}/labels", exist_ok=True)

print(f"✅ Settings configured")
print(f"   Dataset: {BASE_DATASET}")
print(f"   Labeled fraction: {LABELED_FRACTION * 100}%")
print(f"   Warmup epochs: {MIXMATCH_WARMUP_EPOCHS}")
print(f"   SSL epochs: {MIXMATCH_EPOCHS}")
print(f"   Confidence threshold: {MIXMATCH_CONF}")
print(f"   K augmentations: {K_AUGMENTATIONS}")


✅ Settings configured
   Dataset: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8
   Labeled fraction: 30.0%
   Warmup epochs: 60
   SSL epochs: 70
   Confidence threshold: 0.25
   K augmentations: 4


In [6]:
# ============================================================
# CELL 5: DATA SPLIT
# ============================================================
train_images_dir = f"{BASE_DATASET}/train/images"
train_labels_dir = f"{BASE_DATASET}/train/labels"

all_train_imgs = [
    p for p in glob.glob(f"{train_images_dir}/*.*")
    if p.lower().endswith((".jpg", ".jpeg", ".png"))
]

random.seed(42)
random.shuffle(all_train_imgs)

split_idx = int(len(all_train_imgs) * LABELED_FRACTION)
labeled_imgs = all_train_imgs[:split_idx]
unlabeled_imgs = all_train_imgs[split_idx:]

print(f"\n📊 Data split:")
print(f"   Total: {len(all_train_imgs)} images")
print(f"   Labeled ({LABELED_FRACTION*100}%): {len(labeled_imgs)}")
print(f"   Unlabeled ({(1-LABELED_FRACTION)*100}%): {len(unlabeled_imgs)}")

# Copy labeled split
for img_path in tqdm(labeled_imgs, desc="Copying labeled"):
    base = os.path.basename(img_path)
    lbl = base.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt")
    shutil.copy(img_path, f"{MM_LABELED_PATH}/images/{base}")
    lbl_path = f"{train_labels_dir}/{lbl}"
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, f"{MM_LABELED_PATH}/labels/{lbl}")

# Copy unlabeled split
for img_path in tqdm(unlabeled_imgs, desc="Copying unlabeled"):
    base = os.path.basename(img_path)
    lbl = base.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt")
    shutil.copy(img_path, f"{MM_UNLABELED_PATH}/images/{base}")
    lbl_path = f"{train_labels_dir}/{lbl}"
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, f"{MM_UNLABELED_PATH}/labels/{lbl}")

print(f"✅ Data split complete!")



📊 Data split:
   Total: 3408 images
   Labeled (30.0%): 1022
   Unlabeled (70.0%): 2386


Copying unlabeled: 100%|██████████| 2386/2386 [00:16<00:00, 145.37it/s]

✅ Data split complete!


In [7]:
# ============================================================
# CELL 6: IMPROVED AUGMENTATIONS
# ============================================================
weak_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),  # NEW
    A.RandomBrightnessContrast(p=0.3),
    A.Rotate(limit=15, p=0.3),  # NEW
])

strong_aug = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),  # NEW
    A.RandomBrightnessContrast(brightness_limit=0.3, contrast_limit=0.3, p=0.5),
    A.GaussNoise(p=0.3),
    A.Blur(blur_limit=3, p=0.2),
    A.Rotate(limit=20, p=0.4),  # NEW
    A.ColorJitter(p=0.3),  # NEW
])

def write_seg_label_file(out_txt, classes, masks_xy, img_shape):
    h, w = img_shape
    with open(out_txt, "w") as f:
        for cls_id, poly in zip(classes, masks_xy):
            if poly is None or len(poly) < 3:
                continue
            poly = np.array(poly, dtype=np.float32)
            poly[:, 0] = np.clip(poly[:, 0] / w, 0.0, 1.0)
            poly[:, 1] = np.clip(poly[:, 1] / h, 0.0, 1.0)
            coords = " ".join([f"{v:.6f}" for v in poly.reshape(-1)])
            f.write(f"{int(cls_id)} {coords}\n")

print("✅ Augmentations defined")


✅ Augmentations defined


In [8]:
# ============================================================
# CELL 7: WARMUP PHASE
# ============================================================
print("\n" + "="*70)
print("🔥 MIXMATCH WARMUP PHASE")
print("="*70)

MM_DATA_YAML_LABELED = "/kaggle/working/mm_labeled.yaml"
with open(MM_DATA_YAML_LABELED, "w") as f:
    yaml.safe_dump({
        "train": f"{MM_LABELED_PATH}/images",
        "val": f"{BASE_DATASET}/valid/images",
        "test": f"{BASE_DATASET}/test/images",
        "nc": NUM_CLASSES,
        "names": CLASS_NAMES,
    }, f, sort_keys=False)

mm_base_model = YOLO("yolov8m-seg.pt")

print(f"\n🚀 Starting warmup training ({MIXMATCH_WARMUP_EPOCHS} epochs)...")
mm_warmup_results = mm_base_model.train(
    data=MM_DATA_YAML_LABELED,
    epochs=MIXMATCH_WARMUP_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH_SIZE,
    device=DEVICE,
    project="/kaggle/working/mm_runs",
    name="mm_warmup",
    exist_ok=True,
    patience=20,  # INCREASED from 10
    amp=True,
    verbose=False,
    mosaic=0.9,
    mixup=0.05,
)

mm_warmup_best = f"{mm_warmup_results.save_dir}/weights/best.pt"
print(f"✅ Warmup complete! Best: {mm_warmup_best}")



🔥 MIXMATCH WARMUP PHASE

🚀 Starting warmup training (60 epochs)...
Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/mm_labeled.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.05, mode=train, model=yolov8m-seg.pt, momentum=0.937, mosaic=0.9, multi_scale=False, name=mm_warmup, nbs=64, nms=False, opset=None, optimize=False, optimizer

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.577      0.402      0.426      0.214      0.554      0.371      0.381      0.147
Speed: 0.2ms preprocess, 14.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /kaggle/working/mm_runs/mm_warmup
✅ Warmup complete! Best: /kaggle/working/mm_runs/mm_warmup/weights/best.pt


In [10]:
# ============================================================
# CELL 8: IMPROVED PSEUDO-LABEL GENERATION
# ============================================================
print("\n" + "="*70)
print("🏷️ MIXMATCH PSEUDO-LABELING")
print("="*70)

mm_model = YOLO(mm_warmup_best)

unlabeled_imgs_mm = [
    p for p in glob.glob(f"{MM_UNLABELED_PATH}/images/*.*")
    if p.lower().endswith((".jpg", ".jpeg", ".png"))
]

print(f"📝 Pseudo-labeling {len(unlabeled_imgs_mm)} images...")
print(f"   Using {K_AUGMENTATIONS} augmentations, conf ≥ {MIXMATCH_CONF}")
print(f"   Min area filter: {MIN_AREA} pixels")

pseudo_count = 0
skipped_low_conf = 0
skipped_small_area = 0

for img_path in tqdm(unlabeled_imgs_mm, desc="MixMatch pseudo-labels"):
    img = cv2.imread(img_path)
    if img is None:
        continue
    
    # Generate K augmented predictions
    all_predictions = []
    
    for k in range(K_AUGMENTATIONS):
        if k == 0:
            aug = weak_aug(image=img)["image"]
        else:
            aug = strong_aug(image=img)["image"]
        
        results = mm_model.predict(
            source=aug, 
            imgsz=IMGSZ, 
            conf=MIXMATCH_CONF,
            save=False, 
            verbose=False
        )
        
        if results and results[0].masks is not None:
            all_predictions.append(results[0])
    
    if len(all_predictions) == 0:
        skipped_low_conf += 1
        continue
    
    # Use best prediction (highest average confidence)
    best_pred = max(
        all_predictions,
        key=lambda r: r.boxes.conf.mean().item() if len(r.boxes) > 0 else 0
    )
    
    if best_pred.masks is None or len(best_pred.boxes) == 0:
        skipped_low_conf += 1
        continue
    
    # Extract polygons with area filtering
    classes = []
    masks_xy = []
    
    for i in range(len(best_pred.boxes)):
        if best_pred.boxes.conf[i] < MIXMATCH_CONF:
            continue
        
        poly = best_pred.masks.xy[i]
        
        # Handle multiple contours
        if isinstance(poly, list):
            areas = [cv2.contourArea(np.array(p, dtype=np.float32)) for p in poly if len(p) >= 3]
            if len(areas) == 0:
                continue
            poly = poly[int(np.argmax(areas))]
        
        if poly is None or len(poly) < 3:
            continue
        
        # NEW: Area filter
        area = cv2.contourArea(np.array(poly, dtype=np.float32))
        if area < MIN_AREA:
            skipped_small_area += 1
            continue
        
        cls_id = int(best_pred.boxes.cls[i].item()) if hasattr(best_pred.boxes, "cls") else 0
        classes.append(cls_id)
        masks_xy.append(poly)
    
    if len(masks_xy) == 0:
        continue
    
    # Apply strong augmentation for training image
    strong = strong_aug(image=img)["image"]
    
    # Save
    base = os.path.splitext(os.path.basename(img_path))[0]
    out_img = f"{MM_PSEUDO_PATH}/images/{base}.jpg"
    out_lbl = f"{MM_PSEUDO_PATH}/labels/{base}.txt"
    
    write_seg_label_file(out_lbl, classes, masks_xy, strong.shape[:2])
    cv2.imwrite(out_img, strong)
    pseudo_count += 1

print(f"✅ Generated {pseudo_count} pseudo-labeled images")
print(f"   Skipped (low confidence): {skipped_low_conf}")
print(f"   Skipped (small area): {skipped_small_area}")
print(f"   Success rate: {pseudo_count / len(unlabeled_imgs_mm) * 100:.1f}%")



🏷️ MIXMATCH PSEUDO-LABELING
📝 Pseudo-labeling 2386 images...
   Using 4 augmentations, conf ≥ 0.25
   Min area filter: 500 pixels


MixMatch pseudo-labels: 100%|██████████| 2386/2386 [06:03<00:00,  6.57it/s]

✅ Generated 2308 pseudo-labeled images
   Skipped (low confidence): 72
   Skipped (small area): 121
   Success rate: 96.7%


In [11]:
# ============================================================
# CELL 9: MERGE & TRAIN
# ============================================================
print("\n" + "="*70)
print("🎓 MIXMATCH SSL TRAINING")
print("="*70)

# Merge datasets
def merge_dir_mm(src, dst):
    for f in glob.glob(f"{src}/*"):
        shutil.copy(f, f"{dst}/{os.path.basename(f)}")

labeled_count = len(os.listdir(f"{MM_LABELED_PATH}/images"))
pseudo_count = len(os.listdir(f"{MM_PSEUDO_PATH}/images"))

print(f"\n📊 MIXMATCH DATA SUMMARY")
print(f"   Labeled images ({LABELED_FRACTION*100}%): {labeled_count}")
print(f"   Pseudo-labeled (from {(1-LABELED_FRACTION)*100}%): {pseudo_count}")
print(f"   Total for training: {labeled_count + pseudo_count}")

merge_dir_mm(f"{MM_LABELED_PATH}/images", f"{MM_MERGED_PATH}/images")
merge_dir_mm(f"{MM_LABELED_PATH}/labels", f"{MM_MERGED_PATH}/labels")
merge_dir_mm(f"{MM_PSEUDO_PATH}/images", f"{MM_MERGED_PATH}/images")
merge_dir_mm(f"{MM_PSEUDO_PATH}/labels", f"{MM_MERGED_PATH}/labels")

merged_count = len(os.listdir(f"{MM_MERGED_PATH}/images"))
print(f"✅ Merged: {merged_count} images")

# Create YAML
MM_DATA_YAML_MERGED = "/kaggle/working/mm_merged.yaml"
with open(MM_DATA_YAML_MERGED, "w") as f:
    yaml.safe_dump({
        "train": f"{MM_MERGED_PATH}/images",
        "val": f"{BASE_DATASET}/valid/images",
        "test": f"{BASE_DATASET}/test/images",
        "nc": NUM_CLASSES,
        "names": CLASS_NAMES,
    }, f, sort_keys=False)

# Train with improved parameters
gc.collect()
torch.cuda.empty_cache()

mixmatch_model = YOLO(mm_warmup_best)

print(f"\n🚀 Starting MixMatch SSL training...")
print(f"   Epochs: {MIXMATCH_EPOCHS}")
print(f"   Batch: {BATCH_SIZE}")
print(f"   Patience: 20")

mm_results = mixmatch_model.train(
    data=MM_DATA_YAML_MERGED,
    epochs=MIXMATCH_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH_SIZE,
    device=DEVICE,
    project="/kaggle/working/mm_runs",
    name="mm_mixmatch",
    exist_ok=True,
    patience=20,  # INCREASED from 10
    amp=True,
    verbose=False,
    mosaic=0.7,  # INCREASED from 0.5
    mixup=0.15,  # INCREASED from 0.1
    conf=0.001,  # NEW: Lower inference threshold
)

mm_best = f"{mm_results.save_dir}/weights/best.pt"
print(f"\n✅ MixMatch training complete!")
print(f"   Best model: {mm_best}")



🎓 MIXMATCH SSL TRAINING

📊 MIXMATCH DATA SUMMARY
   Labeled images (30.0%): 1022
   Pseudo-labeled (from 70.0%): 2366
   Total for training: 3388
✅ Merged: 3388 images

🚀 Starting MixMatch SSL training...
   Epochs: 70
   Batch: 6
   Patience: 20
Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.001, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/mm_merged.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=70, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.616      0.444      0.471      0.246      0.588        0.4      0.411      0.167
Speed: 0.2ms preprocess, 14.3ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /kaggle/working/mm_runs/mm_mixmatch

✅ MixMatch training complete!
   Best model: /kaggle/working/mm_runs/mm_mixmatch/weights/best.pt


In [12]:
# ============================================================
# CELL 10: VALIDATION
# ============================================================
del mixmatch_model
gc.collect()
torch.cuda.empty_cache()

mm_final = YOLO(mm_best)
mm_val_metrics = mm_final.val(split="test", verbose=False)

print(f"\n{'='*70}")
print(f"MIXMATCH RESULTS")
print(f"{'='*70}")
print(f"   Mask mAP@0.5: {mm_val_metrics.seg.map50:.4f}")
print(f"   Mask mAP@0.5:0.95: {mm_val_metrics.seg.map:.4f}")
print(f"   Box mAP@0.5: {mm_val_metrics.box.map50:.4f}")
print(f"{'='*70}")
print(f"\n✅ MixMatch complete!")


Ultralytics 8.3.236 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m-seg summary (fused): 105 layers, 27,222,963 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 7.3±3.0 MB/s, size: 34.6 KB)
val: Scanning /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/test/labels... 487 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 487/487 234.4it/s 2.1s0.1s
WARNING ⚠️ val: Cache directory /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/test is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 31/31 1.6it/s 18.9s0.6s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        487        958      0.635      0.471      0.501      0.248      0.611      0.416       0.43      0.173
Speed: 1.5ms preprocess, 29.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to /kaggle/working/runs/segment/val

MIXMATCH RESULTS
   Mask mAP@0.5: 0.4302
   Mask mAP@0.5:0.95: 0.1728
   Box mAP@0.5: 0.5007

✅ MixMatch complete!


In [1]:
!pip -q install ultralytics albumentations --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.2 MB/s eta 0:00:00


In [2]:
%%writefile /kaggle/working/data.yaml
# ============================================================
# YOLOv12 Dataset Configuration for Instance Segmentation
# ============================================================

train: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/train/images
val: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/valid/images
test: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/test/images

# Number of classes
nc: 1

# Class names
names: ['car-damage']

Writing /kaggle/working/data.yaml


In [3]:
from ultralytics import YOLO
import albumentations as A
import os, glob, random, shutil, yaml, cv2
import numpy as np
from tqdm import tqdm
import torch
import matplotlib.pyplot as plt


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [4]:
# ============================================================
# 0) Paths & constants
# ============================================================
BASE_DATASET = "/kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8"   # dataset root (train/valid/test)
DATA_YAML    = "/kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/data.yamlD"           # existing baseline yaml (optional)

FM_LABELED_PATH   = "/kaggle/working/fm_labeled"
FM_UNLABELED_PATH = "/kaggle/working/fm_unlabeled"
FM_PSEUDO_PATH    = "/kaggle/working/fm_pseudo"      # strong-view pseudo-labeled images
FM_MERGED_PATH    = "/kaggle/working/fm_merged"

for p in [FM_LABELED_PATH, FM_UNLABELED_PATH, FM_PSEUDO_PATH, FM_MERGED_PATH]:
    os.makedirs(f"{p}/images", exist_ok=True)
    os.makedirs(f"{p}/labels", exist_ok=True)

NUM_CLASSES = 1
CLASS_NAMES = ["car-damage"]

LABELED_FRACTION = 0.2      # 20% labeled, 80% unlabeled
WARMUP_EPOCHS    = 30       # supervised-only epochs
FIXMATCH_EPOCHS  = 50       # training on labeled + pseudo-labeled
IMGSZ            = 640
BATCH            = 16
DEVICE           = 0        # GPU id or "cpu"
PSEUDO_CONF_TH   = 0.7      # FixMatch-style high confidence threshold
MAX_PSEUDO       = None     # e.g. 500 to cap for speed

In [5]:
# ============================================================
# 1) Weak & Strong augmentations (FixMatch style)
#    Weak: minimal transform (reliable prediction)
#    Strong: same geometry + heavier color/blur/noise
# ============================================================
weak_aug = A.Compose([
    A.Resize(IMGSZ, IMGSZ),
])

strong_aug = A.Compose([
    A.Resize(IMGSZ, IMGSZ),
    A.ColorJitter(p=0.5),
    A.RandomBrightnessContrast(p=0.5),
    A.GaussNoise(p=0.4),
    A.MotionBlur(p=0.3),
])

In [6]:
# ============================================================
# 2) Split dataset into labeled / unlabeled
# ============================================================
all_train_imgs = [
    p for p in glob.glob(f"{BASE_DATASET}/train/images/*.*")
    if p.lower().endswith((".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"))
]
random.shuffle(all_train_imgs)

split_idx = int(len(all_train_imgs) * LABELED_FRACTION)
labeled_imgs   = all_train_imgs[:split_idx]
unlabeled_imgs = all_train_imgs[split_idx:]

def safe_copy_pairs(image_paths, dst_root):
    """Copy images and their YOLO label .txt (if exists) to dst_root/images, dst_root/labels."""
    os.makedirs(f"{dst_root}/images", exist_ok=True)
    os.makedirs(f"{dst_root}/labels", exist_ok=True)
    for img in image_paths:
        base = os.path.splitext(os.path.basename(img))[0]
        lbl  = img.replace("/images/", "/labels/").rsplit(".", 1)[0] + ".txt"
        shutil.copy(img, f"{dst_root}/images/{os.path.basename(img)}")
        if os.path.exists(lbl):
            shutil.copy(lbl, f"{dst_root}/labels/{base}.txt")

safe_copy_pairs(labeled_imgs,   FM_LABELED_PATH)
safe_copy_pairs(unlabeled_imgs, FM_UNLABELED_PATH)

print(f"✅ FixMatch split → Labeled: {len(labeled_imgs)} | Unlabeled: {len(unlabeled_imgs)}")


✅ FixMatch split → Labeled: 681 | Unlabeled: 2727


In [7]:
# quick integrity check
lab_img_names = {os.path.splitext(f)[0] for f in os.listdir(f"{FM_LABELED_PATH}/images")}
lab_lbl_names = {os.path.splitext(f)[0] for f in os.listdir(f"{FM_LABELED_PATH}/labels")}
print(f"🧩 Integrity (labeled): images={len(lab_img_names)} labels={len(lab_lbl_names)}  missing_labels={len(lab_img_names - lab_lbl_names)}")

🧩 Integrity (labeled): images=681 labels=681  missing_labels=0


In [8]:
# ============================================================
# 3) Build labeled YAML for warmup supervised training
# ============================================================
FM_DATA_YAML_LABELED = "/kaggle/working/fm_labeled.yaml"
with open(FM_DATA_YAML_LABELED, "w") as f:
    yaml.safe_dump(
        {
            "train": f"{FM_LABELED_PATH}/images",
            "val":   f"{BASE_DATASET}/valid/images",
            "test":  f"{BASE_DATASET}/test/images",
            "nc": NUM_CLASSES,
            "names": CLASS_NAMES,
        },
        f,
        sort_keys=False,
    )

print("🔧 Labeled YAML:\n", open(FM_DATA_YAML_LABELED).read())

🔧 Labeled YAML:
 train: /kaggle/working/fm_labeled/images
val: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/valid/images
test: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/test/images
nc: 1
names:
- car-damage



In [9]:
# ============================================================
# 4) Warmup: train base model on labeled subset
# ============================================================
base_model = YOLO("yolov8s-seg.pt")
warmup_results = base_model.train(
    data=FM_DATA_YAML_LABELED,
    epochs=WARMUP_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0,
    project="/kaggle/working/fm_runs",
    name="fm_warmup_teacher",
    exist_ok=True,
    patience=10,
    amp=True,
)
warmup_best = f"{warmup_results.save_dir}/weights/best.pt"
print("✅ Warmup best checkpoint:", warmup_best)

# Reload best warmup weights (FixMatch uses same network as "teacher" & "student")
model = YOLO(warmup_best) 


Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/fm_labeled.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fm_warmup_teacher, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=10, perspective=0.0, plot

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.473      0.354      0.349      0.164      0.429      0.324      0.295      0.106
Speed: 0.2ms preprocess, 6.0ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to /kaggle/working/fm_runs/fm_warmup_teacher
✅ Warmup best checkpoint: /kaggle/working/fm_runs/fm_warmup_teacher/weights/best.pt


In [10]:

# ============================================================
# 5) FixMatch pseudo-labeling:
#    weak(x_u) → pseudo-label, strong(x_u) used for training
# ============================================================
to_label = unlabeled_imgs if MAX_PSEUDO is None else unlabeled_imgs[:MAX_PSEUDO]
print(f"📝 FixMatch pseudo-labeling {len(to_label)} unlabeled images (conf ≥ {PSEUDO_CONF_TH})")

def write_seg_label_file(out_txt, classes, masks_xy, img_shape):
    """
    Write YOLO segmentation polygons to txt:
    <cls> x1 y1 x2 y2 ... (normalized to [0,1]).
    """
    h, w = img_shape  # (H, W)
    with open(out_txt, "w") as f:
        for cls_id, poly in zip(classes, masks_xy):
            if poly is None or len(poly) < 3:
                continue
            poly = np.array(poly, dtype=np.float32)
            poly[:, 0] = np.clip(poly[:, 0] / w, 0.0, 1.0)
            poly[:, 1] = np.clip(poly[:, 1] / h, 0.0, 1.0)
            coords = " ".join([f"{v:.6f}" for v in poly.reshape(-1)])
            f.write(f"{int(cls_id)} {coords}\n")

for img_path in tqdm(to_label, desc="FixMatch pseudo-labels"):
    img = cv2.imread(img_path)
    if img is None:
        continue

    # --- Weak view: used for prediction (pseudo-labels) ---
    weak = weak_aug(image=img)["image"]

    # YOLO expects BGR uint8; weak is already resized
    # Run prediction on weak view
    results = model.predict(source=weak, imgsz=IMGSZ, conf=PSEUDO_CONF_TH, save=False, verbose=False)
    if not results:
        continue
    r = results[0]
    if r.masks is None or len(r.boxes) == 0:
        continue

    # Filter by confidence, like FixMatch's "high confidence" rule
    confs = r.boxes.conf.cpu().numpy()
    keep_idx = np.where(confs >= PSEUDO_CONF_TH)[0]
    if keep_idx.size == 0:
        continue

    # Extract polygons & class ids for kept instances
    classes = []
    masks_xy = []
    for i in keep_idx:
        if r.masks is None or r.masks.xy is None:
            continue
        polys = r.masks.xy[i]
        if isinstance(polys, list):
            # choose largest polygon by area
            areas = [cv2.contourArea(np.array(p, dtype=np.float32)) for p in polys if len(p) >= 3]
            if len(areas) == 0:
                continue
            poly = polys[int(np.argmax(areas))]
        else:
            poly = polys
        if poly is None or len(poly) < 3:
            continue
        cls_id = int(r.boxes.cls[i].item()) if hasattr(r.boxes, "cls") else 0
        classes.append(cls_id)
        masks_xy.append(poly)

    if len(masks_xy) == 0:
        continue

    # --- Strong view: used for actual training image in FixMatch ---
    strong = strong_aug(image=img)["image"]

    # Save strong-view image + label with pseudo-labels from weak view
    base = os.path.splitext(os.path.basename(img_path))[0]
    out_img = f"{FM_PSEUDO_PATH}/images/{base}.jpg"
    out_lbl = f"{FM_PSEUDO_PATH}/labels/{base}.txt"

    # Ensure shape is IMGSZ x IMGSZ
    h, w = strong.shape[:2]
    assert h == IMGSZ and w == IMGSZ, "Strong view must be resized to IMGSZ"

    write_seg_label_file(out_lbl, classes, masks_xy, (h, w))
    cv2.imwrite(out_img, strong)

print("✅ FixMatch pseudo-label generation complete.")


📝 FixMatch pseudo-labeling 2727 unlabeled images (conf ≥ 0.7)


FixMatch pseudo-labels: 100%|██████████| 2727/2727 [00:51<00:00, 52.94it/s]

✅ FixMatch pseudo-label generation complete.


In [11]:
# ============================================================
# 6) Merge labeled data + FixMatch pseudo-labeled strong views
# ============================================================
def merge_dir(src, dst):
    for f in glob.glob(f"{src}/*"):
        shutil.copy(f, f"{dst}/{os.path.basename(f)}")

merge_dir(f"{FM_LABELED_PATH}/images", f"{FM_MERGED_PATH}/images")
merge_dir(f"{FM_LABELED_PATH}/labels", f"{FM_MERGED_PATH}/labels")
merge_dir(f"{FM_PSEUDO_PATH}/images",  f"{FM_MERGED_PATH}/images")
merge_dir(f"{FM_PSEUDO_PATH}/labels",  f"{FM_MERGED_PATH}/labels")

FM_DATA_YAML_MERGED = "/kaggle/working/fm_merged.yaml"
with open(FM_DATA_YAML_MERGED, "w") as f:
    yaml.safe_dump(
        {
            "train": f"{FM_MERGED_PATH}/images",
            "val":   f"{BASE_DATASET}/valid/images",
            "test":  f"{BASE_DATASET}/test/images",
            "nc": NUM_CLASSES,
            "names": CLASS_NAMES,
        },
        f,
        sort_keys=False,
    )

print("🔧 Merged YAML:\n", open(FM_DATA_YAML_MERGED).read())

🔧 Merged YAML:
 train: /kaggle/working/fm_merged/images
val: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/valid/images
test: /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/test/images
nc: 1
names:
- car-damage



In [12]:
# ============================================================
# 7) Train model on merged dataset (labeled + FixMatch pseudo)
# ============================================================
fixmatch_model = YOLO(warmup_best)
fm_results = fixmatch_model.train(
    data=FM_DATA_YAML_MERGED,
    epochs=FIXMATCH_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH,
    device=0,
    project="/kaggle/working/fm_runs",
    name="fm_fixmatch_student",
    exist_ok=True,
    patience=10,
    amp=True,
)
fm_best = f"{fm_results.save_dir}/weights/best.pt"
print("✅ FixMatch student best checkpoint:", fm_best)

# (Optional) Validate
_ = fixmatch_model.val(split="val")

Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/fm_merged.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=/kaggle/working/fm_runs/fm_warmup_teacher/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=fm_fixmatch_student, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_m

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.633      0.421      0.462      0.238      0.615       0.39      0.413      0.171
Speed: 0.2ms preprocess, 6.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to /kaggle/working/fm_runs/fm_fixmatch_student
✅ FixMatch student best checkpoint: /kaggle/working/fm_runs/fm_fixmatch_student/weights/best.pt
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 36.1±20.5 MB/s, size: 35.5 KB)
val: Scanning /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/valid/labels... 974 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 974/974 689.8it/s 1.4s
WARNING ⚠️ val: Cache directory /kaggle/input/car-damage-detection/Car-Damage detection.v1i.yolov8/valid is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mA

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.637      0.421      0.462      0.237      0.615       0.39      0.413      0.171
Speed: 1.1ms preprocess, 12.8ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to /kaggle/working/runs/segment/val


In [13]:
# ============================================================
# FIND BEST.PT FILES
# ============================================================
import os
import glob

print("🔍 Searching for best.pt files...\n")

# Method 1: Search in fm_runs
for filepath in glob.glob("/kaggle/working/fm_runs/**/best.pt", recursive=True):
    size = os.path.getsize(filepath) / (1024*1024)
    print(f"✅ Found: {filepath}")
    print(f"   Size: {size:.1f} MB\n")

# Method 2: Show directory structure
print("📁 Directory structure:")
for root, dirs, files in os.walk("/kaggle/working/fm_runs"):
    level = root.replace("/kaggle/working/fm_runs", "").count(os.sep)
    indent = " " * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = " " * 2 * (level + 1)
    for file in files[:5]:  # Show first 5 files
        if file.endswith(".pt"):
            print(f"{subindent}📄 {file}")


🔍 Searching for best.pt files...

✅ Found: /kaggle/working/fm_runs/fm_warmup_teacher/weights/best.pt
   Size: 22.7 MB

✅ Found: /kaggle/working/fm_runs/fm_fixmatch_student/weights/best.pt
   Size: 22.7 MB

📁 Directory structure:
fm_runs/
  fm_warmup_teacher/
    weights/
      📄 best.pt
      📄 last.pt
  fm_fixmatch_student/
    weights/
      📄 best.pt
      📄 last.pt


In [14]:
# ============================================================
# COPY BEST MODELS TO ROOT FOR EASY ACCESS
# ============================================================
import shutil
import os

print("\n" + "="*70)
print("📦 COPYING BEST MODELS TO ROOT")
print("="*70)

# Find and copy FixMatch best model
fm_best_path = None
for root, dirs, files in os.walk("/kaggle/working/fm_runs"):
    if "best.pt" in files:
        fm_best_path = os.path.join(root, "best.pt")
        break

if fm_best_path:
    shutil.copy(fm_best_path, "/kaggle/working/fixmatch_best.pt")
    print(f"✅ FixMatch: {fm_best_path}")
    print(f"   → Copied to: /kaggle/working/fixmatch_best.pt")
else:
    print("❌ FixMatch best.pt not found!")

# List all files in root
print(f"\n📁 Files in /kaggle/working/:")
for item in os.listdir("/kaggle/working"):
    if item.endswith(".pt"):
        size = os.path.getsize(f"/kaggle/working/{item}") / (1024*1024)
        print(f"   {item:30s} ({size:.1f} MB)")

print("\n✅ Best model copied! Download 'fixmatch_best.pt' from Output panel.")



📦 COPYING BEST MODELS TO ROOT
✅ FixMatch: /kaggle/working/fm_runs/fm_warmup_teacher/weights/best.pt
   → Copied to: /kaggle/working/fixmatch_best.pt

📁 Files in /kaggle/working/:
   fixmatch_best.pt               (22.7 MB)
   yolov8s-seg.pt                 (22.8 MB)
   yolo11n.pt                     (5.4 MB)

✅ Best model copied! Download 'fixmatch_best.pt' from Output panel.


In [2]:
!pip -q install ultralytics --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.5 MB/s eta 0:00:00a 0:00:01


In [3]:
%%writefile /kaggle/working/data.yaml
# ============================================================
# YOLOv12 Dataset Configuration for Instance Segmentation
# ============================================================

train: /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/train/images
val: /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/valid/images
test: /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/test/images
# Number of classes
nc: 1

# Class names
names: ['Car-Damage']

Overwriting /kaggle/working/data.yaml


In [4]:
# ============================================================
# CELL 2: IMPORTS & SETUP
# ============================================================
import os
import glob
import random
import shutil
import yaml
import cv2
import numpy as np
from pathlib import Path
from tqdm import tqdm
import gc
import torch

from ultralytics import YOLO

print("\n" + "="*70)
print("MEAN TEACHER - SEMI-SUPERVISED INSTANCE SEGMENTATION")
print("="*70)
print("✅ All imports successful!")

# Verify CUDA
print(f"\n🔧 System Info:")
print(f"   Python: {torch.__version__}")
print(f"   PyTorch: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.

MEAN TEACHER - SEMI-SUPERVISED INSTANCE SEGMENTATION
✅ All imports successful!

🔧 System Info:
   Python: 2.6.0+cu124
   PyTorch: 2.6.0+cu124
   CUDA available: True
   GPU: Tesla T4


In [5]:
# ============================================================
# CELL 3: CONFIGURATION SETTINGS
# ============================================================
print("\n" + "="*70)
print("CONFIGURATION")
print("="*70)

# Dataset paths
BASE_DATASET = "/kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8"
NUM_CLASSES = 1
CLASS_NAMES = ["Car-Damage"]

# Model settings
IMGSZ = 640
DEVICE = 0  # Single GPU

# Data split
LABELED_FRACTION = 0.2  # 20% labeled, 80% unlabeled

# Mean Teacher hyperparameters
MT_WARMUP_EPOCHS = 40        # Warmup on labeled only
MEAN_TEACHER_EPOCHS = 50     # SSL training epochs
BATCH_SIZE = 4               # Per-image batch size
MT_CONF = 0.6                # Confidence threshold for pseudo-labels

# Pseudo-label settings
PSEUDO_PERCENTAGE = 0.15     # Use 15% of unlabeled per epoch
# PSEUDO_BATCH_SIZE will be calculated after data split

# Working directories
MT_LABELED_PATH = "/kaggle/working/mt_labeled"
MT_UNLABELED_PATH = "/kaggle/working/mt_unlabeled"
MT_PSEUDO_PATH = "/kaggle/working/mt_pseudo"
MT_MERGED_PATH = "/kaggle/working/mt_merged"

print(f"\n📊 Settings:")
print(f"   Model: YOLOv8s-seg")
print(f"   Image size: {IMGSZ}")
print(f"   Device: GPU {DEVICE}")
print(f"   Labeled fraction: {LABELED_FRACTION*100}%")
print(f"   Warmup epochs: {MT_WARMUP_EPOCHS}")
print(f"   SSL epochs: {MEAN_TEACHER_EPOCHS}")
print(f"   Batch size: {BATCH_SIZE}")
print(f"   Confidence threshold: {MT_CONF}")
print(f"   Pseudo-label percentage: {PSEUDO_PERCENTAGE*100}%")



CONFIGURATION

📊 Settings:
   Model: YOLOv8s-seg
   Image size: 640
   Device: GPU 0
   Labeled fraction: 20.0%
   Warmup epochs: 40
   SSL epochs: 50
   Batch size: 4
   Confidence threshold: 0.6
   Pseudo-label percentage: 15.0%


In [6]:
# ============================================================
# CELL 4: CREATE DIRECTORY STRUCTURE
# ============================================================
print("\n" + "="*70)
print("CREATING DIRECTORIES")
print("="*70)

for path in [MT_LABELED_PATH, MT_UNLABELED_PATH, MT_PSEUDO_PATH, MT_MERGED_PATH]:
    os.makedirs(f"{path}/images", exist_ok=True)
    os.makedirs(f"{path}/labels", exist_ok=True)
    print(f"✅ {path}")

print("\n✅ All directories created!")



CREATING DIRECTORIES
✅ /kaggle/working/mt_labeled
✅ /kaggle/working/mt_unlabeled
✅ /kaggle/working/mt_pseudo
✅ /kaggle/working/mt_merged

✅ All directories created!


In [7]:
# ============================================================
# CELL 5: HELPER FUNCTION FOR SEGMENTATION LABELS
# ============================================================
def write_seg_label_file(out_txt, classes, masks_xy, img_shape):
    """
    Write YOLO segmentation format label file.
    
    Args:
        out_txt: Output label file path
        classes: List of class IDs
        masks_xy: List of polygon coordinates (unnormalized)
        img_shape: (height, width) of the image
    
    Format: <class_id> <x1> <y1> <x2> <y2> ... (normalized 0-1)
    """
    h, w = img_shape
    with open(out_txt, "w") as f:
        for cls_id, poly in zip(classes, masks_xy):
            if poly is None or len(poly) < 3:
                continue
            
            poly = np.array(poly, dtype=np.float32)
            
            # Normalize coordinates to [0, 1]
            poly[:, 0] = np.clip(poly[:, 0] / w, 0.0, 1.0)
            poly[:, 1] = np.clip(poly[:, 1] / h, 0.0, 1.0)
            
            # Flatten polygon to string
            coords = " ".join([f"{v:.6f}" for v in poly.reshape(-1)])
            
            # Write: class_id x1 y1 x2 y2 ...
            f.write(f"{int(cls_id)} {coords}\n")

print("✅ Helper function defined")


✅ Helper function defined


In [8]:
# ============================================================
# CELL 6: DATA SPLIT (20% LABELED, 80% UNLABELED)
# ============================================================
print("\n" + "="*70)
print("DATA SPLITTING")
print("="*70)

train_images_dir = f"{BASE_DATASET}/train/images"
train_labels_dir = f"{BASE_DATASET}/train/labels"

# Get all training images
all_train_imgs = [
    p for p in glob.glob(f"{train_images_dir}/*.*")
    if p.lower().endswith((".jpg", ".jpeg", ".png"))
]

print(f"\n📦 Total training images: {len(all_train_imgs)}")

# Shuffle with fixed seed for reproducibility
random.seed(42)
random.shuffle(all_train_imgs)

# Split into labeled (20%) and unlabeled (80%)
split_idx = int(len(all_train_imgs) * LABELED_FRACTION)
labeled_imgs = all_train_imgs[:split_idx]
unlabeled_imgs = all_train_imgs[split_idx:]

print(f"\n📊 Split results:")
print(f"   Labeled (20%):   {len(labeled_imgs)} images")
print(f"   Unlabeled (80%): {len(unlabeled_imgs)} images")

# Calculate PSEUDO_BATCH_SIZE dynamically
PSEUDO_BATCH_SIZE = int(len(unlabeled_imgs) * PSEUDO_PERCENTAGE)
print(f"\n📊 Pseudo-label strategy:")
print(f"   Using per epoch: {PSEUDO_BATCH_SIZE} ({PSEUDO_PERCENTAGE*100}%)")

# Copy labeled split
print(f"\n📁 Copying labeled split...")
for img_path in tqdm(labeled_imgs, desc="Labeled images"):
    base = os.path.basename(img_path)
    lbl = base.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt")
    
    # Copy image
    shutil.copy(img_path, f"{MT_LABELED_PATH}/images/{base}")
    
    # Copy label if exists
    lbl_path = f"{train_labels_dir}/{lbl}"
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, f"{MT_LABELED_PATH}/labels/{lbl}")

# Copy unlabeled split
print(f"\n📁 Copying unlabeled split...")
for img_path in tqdm(unlabeled_imgs, desc="Unlabeled images"):
    base = os.path.basename(img_path)
    lbl = base.replace(".jpg", ".txt").replace(".jpeg", ".txt").replace(".png", ".txt")
    
    # Copy image
    shutil.copy(img_path, f"{MT_UNLABELED_PATH}/images/{base}")
    
    # Copy label (for reference, not used in training)
    lbl_path = f"{train_labels_dir}/{lbl}"
    if os.path.exists(lbl_path):
        shutil.copy(lbl_path, f"{MT_UNLABELED_PATH}/labels/{lbl}")

print(f"\n✅ Data split complete!")
print(f"   Labeled: {len(os.listdir(f'{MT_LABELED_PATH}/images'))} images")
print(f"   Unlabeled: {len(os.listdir(f'{MT_UNLABELED_PATH}/images'))} images")



DATA SPLITTING

📦 Total training images: 3408

📊 Split results:
   Labeled (20%):   681 images
   Unlabeled (80%): 2727 images

📊 Pseudo-label strategy:
   Using per epoch: 409 (15.0%)

📁 Copying labeled split...


Labeled images: 100%|██████████| 681/681 [00:09<00:00, 68.30it/s]



📁 Copying unlabeled split...


Unlabeled images: 100%|██████████| 2727/2727 [00:37<00:00, 71.94it/s]


✅ Data split complete!
   Labeled: 681 images
   Unlabeled: 2727 images


In [9]:
# ============================================================
# CELL 7: WARMUP PHASE - TRAIN ON LABELED DATA
# ============================================================
print("\n" + "="*70)
print("PHASE 1: WARMUP TRAINING (SUPERVISED)")
print("="*70)

# Create YAML for labeled data only
MT_DATA_YAML_LABELED = "/kaggle/working/mt_labeled.yaml"
with open(MT_DATA_YAML_LABELED, "w") as f:
    yaml.safe_dump({
        "train": f"{MT_LABELED_PATH}/images",
        "val": f"{BASE_DATASET}/valid/images",
        "test": f"{BASE_DATASET}/test/images",
        "nc": NUM_CLASSES,
        "names": CLASS_NAMES,
    }, f, sort_keys=False)

print(f"\n✅ Created: {MT_DATA_YAML_LABELED}")

# Load pretrained YOLOv8s-seg model
print(f"\n📦 Loading YOLOv8s-seg...")
mt_base_model = YOLO("yolov8s-seg.pt")
print(f"✅ Model loaded")

# Train warmup model
print(f"\n🔥 Starting warmup training...")
print(f"   Epochs: {MT_WARMUP_EPOCHS}")
print(f"   Data: Labeled only ({len(labeled_imgs)} images)")
print(f"   Batch size: {BATCH_SIZE}")

mt_warmup_results = mt_base_model.train(
    data=MT_DATA_YAML_LABELED,
    epochs=MT_WARMUP_EPOCHS,
    imgsz=IMGSZ,
    batch=BATCH_SIZE,
    device=DEVICE,
    project="/kaggle/working/mt_runs",
    name="mt_warmup_student",
    exist_ok=True,
    patience=10,
    amp=True,
    cache=False,
    verbose=True,
)

MT_WARMUP_BEST = f"{mt_warmup_results.save_dir}/weights/best.pt"
print(f"\n✅ Warmup complete!")
print(f"   Best model: {MT_WARMUP_BEST}")





PHASE 1: WARMUP TRAINING (SUPERVISED)

✅ Created: /kaggle/working/mt_labeled.yaml

📦 Loading YOLOv8s-seg...
✅ Model loaded

🔥 Starting warmup training...
   Epochs: 40
   Data: Labeled only (681 images)
   Batch size: 4
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/mt_labeled.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, m

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.416      0.343       0.32      0.142       0.42      0.294      0.273     0.0972
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 2.5ms postprocess per image
Results saved to /kaggle/working/mt_runs/mt_warmup_student

✅ Warmup complete!
   Best model: /kaggle/working/mt_runs/mt_warmup_student/weights/best.pt


In [15]:
# ============================================================
# CELL 8: MEAN TEACHER TRAINING LOOP
# ============================================================
print("\n" + "="*70)
print("PHASE 2: MEAN TEACHER SSL TRAINING")
print("="*70)

# Get unlabeled image paths
unlabeled_imgs_list = [
    p for p in glob.glob(f"{MT_UNLABELED_PATH}/images/*.*")
    if p.lower().endswith((".jpg", ".jpeg", ".png"))
]

print(f"\n📦 Available unlabeled images: {len(unlabeled_imgs_list)}")
print(f"   Using per epoch: {PSEUDO_BATCH_SIZE}")
print(f"   Confidence threshold: {MT_CONF}")

# Track best model
best_map = 0.0
best_epoch = 0
best_model_path = "/kaggle/working/mt_best.pt"


# ============================================================
# TRACKING SETUP - Add this before the training loop
# ============================================================
import pandas as pd
import matplotlib.pyplot as plt

# Initialize tracking dictionary
mt_metrics_history = {
    'epoch': [],
    'train/box_loss': [],
    'train/seg_loss': [],
    'train/cls_loss': [],
    'train/dfl_loss': [],
    'val/box_loss': [],
    'val/seg_loss': [],
    'val/cls_loss': [],
    'val/dfl_loss': [],
    'metrics/precision(B)': [],
    'metrics/recall(B)': [],
    'metrics/mAP50(B)': [],
    'metrics/mAP50-95(B)': [],
    'metrics/precision(M)': [],
    'metrics/recall(M)': [],
    'metrics/mAP50(M)': [],
    'metrics/mAP50-95(M)': [],
}

print("✅ Metrics tracking initialized")



# Training loop
for epoch in range(MEAN_TEACHER_EPOCHS):
    print("\n" + "="*70)
    print(f"EPOCH {epoch+1}/{MEAN_TEACHER_EPOCHS}")
    print("="*70)
    
    # ============================================================
    # Step 1: Load Models (Student and Teacher)
    # ============================================================
    print(f"\n📦 Loading models...")
    
    if epoch == 0:
        # First epoch: Both use warmup weights
        student_model = YOLO(MT_WARMUP_BEST)
        teacher_model = YOLO(MT_WARMUP_BEST)
        print(f"   Using warmup weights")
    else:
        # Later epochs: Use previous epoch weights
        prev_checkpoint = f"/kaggle/working/mt_epoch_{epoch-1}.pt"
        if os.path.exists(prev_checkpoint):
            student_model = YOLO(prev_checkpoint)
            teacher_model = YOLO(prev_checkpoint)
            print(f"   Using epoch {epoch-1} weights")
        else:
            student_model = YOLO(MT_WARMUP_BEST)
            teacher_model = YOLO(MT_WARMUP_BEST)
            print(f"   Warning: Previous epoch not found, using warmup")
    
    print(f"✅ Models loaded")
    
    # ============================================================
    # Step 2: Generate Pseudo-Labels with Teacher
    # ============================================================
    print(f"\n🏷️  Generating pseudo-labels with teacher model...")
    
    # Clear previous pseudo-labels
    for f in glob.glob(f"{MT_PSEUDO_PATH}/images/*"):
        os.remove(f)
    for f in glob.glob(f"{MT_PSEUDO_PATH}/labels/*"):
        os.remove(f)
    
    # Sample subset of unlabeled images
    sample_size = min(PSEUDO_BATCH_SIZE, len(unlabeled_imgs_list))
    sample_imgs = random.sample(unlabeled_imgs_list, sample_size)
    
    pseudo_count = 0
    
    for img_path in tqdm(sample_imgs, desc=f"Teacher (conf≥{MT_CONF})"):
        # Read image
        img = cv2.imread(img_path)
        if img is None:
            continue
        
        # Teacher prediction
        results = teacher_model.predict(
            source=img,
            imgsz=IMGSZ,
            conf=MT_CONF,
            save=False,
            verbose=False,
        )
        
        # Check if valid predictions
        if not results or results[0].masks is None or len(results[0].boxes) == 0:
            continue
        
        r = results[0]
        classes = []
        masks_xy = []
        
        # Extract high-confidence masks
        for i in range(len(r.boxes)):
            if r.boxes.conf[i] < MT_CONF:
                continue
            
            # Get polygon
            poly = r.masks.xy[i]
            
            # Handle multi-part polygons (keep largest)
            if isinstance(poly, list):
                areas = [cv2.contourArea(np.array(p, dtype=np.float32))
                        for p in poly if len(p) >= 3]
                if not areas:
                    continue
                poly = poly[int(np.argmax(areas))]
            
            # Validate polygon
            if poly is None or len(poly) < 3:
                continue
            
            # Get class ID
            cls_id = int(r.boxes.cls[i].item()) if hasattr(r.boxes, "cls") else 0
            
            classes.append(cls_id)
            masks_xy.append(poly)
        
        # Skip if no valid masks
        if not masks_xy:
            continue
        
        # Save pseudo-labeled image and label
        base = os.path.splitext(os.path.basename(img_path))[0]
        out_img = f"{MT_PSEUDO_PATH}/images/{base}.jpg"
        out_lbl = f"{MT_PSEUDO_PATH}/labels/{base}.txt"
        
        # Write label file
        write_seg_label_file(out_lbl, classes, masks_xy, img.shape[:2])
        
        # Copy image
        shutil.copy(img_path, out_img)
        
        pseudo_count += 1
    
    print(f"\n   ✅ Generated {pseudo_count} pseudo-labeled images")
    
    # ============================================================
    # Step 3: Merge Labeled + Pseudo-Labeled Data
    # ============================================================
    print(f"\n📦 Merging datasets...")
    
    # Clear merged directory
    for f in glob.glob(f"{MT_MERGED_PATH}/images/*"):
        os.remove(f)
    for f in glob.glob(f"{MT_MERGED_PATH}/labels/*"):
        os.remove(f)
    
    # Copy labeled data
    for src in [f"{MT_LABELED_PATH}/images", f"{MT_PSEUDO_PATH}/images"]:
        for f in glob.glob(f"{src}/*"):
            shutil.copy(f, f"{MT_MERGED_PATH}/images/{os.path.basename(f)}")
    
    for src in [f"{MT_LABELED_PATH}/labels", f"{MT_PSEUDO_PATH}/labels"]:
        for f in glob.glob(f"{src}/*"):
            shutil.copy(f, f"{MT_MERGED_PATH}/labels/{os.path.basename(f)}")
    
    labeled_count = len(os.listdir(f"{MT_LABELED_PATH}/images"))
    total_count = len(os.listdir(f"{MT_MERGED_PATH}/images"))
    
    print(f"   Labeled: {labeled_count}")
    print(f"   Pseudo-labeled: {pseudo_count}")
    print(f"   Total: {total_count}")
    print(f"✅ Datasets merged")
    
    # Create YAML for merged data
    yaml_path = "/kaggle/working/mt_merged.yaml"
    with open(yaml_path, "w") as f:
        yaml.safe_dump({
            "train": f"{MT_MERGED_PATH}/images",
            "val": f"{BASE_DATASET}/valid/images",
            "test": f"{BASE_DATASET}/test/images",
            "nc": NUM_CLASSES,
            "names": CLASS_NAMES,
        }, f, sort_keys=False)
    
    # ============================================================
    # Step 4: Train Student on Labeled + Pseudo-Labeled
    # ============================================================
    print(f"\n🎓 Training student model...")
    print(f"   Training data: {total_count} images")
    print(f"   Epochs: 1 (incremental)")
    
    train_result = student_model.train(
        data=yaml_path,
        epochs=1,
        imgsz=IMGSZ,
        batch=BATCH_SIZE,
        device=DEVICE,
        project="/kaggle/working/mt_runs",
        name=f"mt_epoch_{epoch}",
        exist_ok=True,
        resume=False,
        amp=True,
        verbose=False,
        cache=False,
    )
    
    # Save epoch checkpoint
    epoch_checkpoint = f"/kaggle/working/mt_epoch_{epoch}.pt"
    student_model.save(epoch_checkpoint)
    print(f"✅ Checkpoint saved: {epoch_checkpoint}")
    
       # ============================================================
    # Step 5: Validate Student Model
    # ============================================================
    print(f"\n📊 Validating student model...")
    
    val_result = student_model.val(split="val", verbose=False)
    
    # Get metrics
    current_map50 = val_result.seg.map50
    current_map = val_result.seg.map
    
    print(f"\n   📈 Metrics:")
    print(f"      Mask mAP@0.5:      {current_map50:.4f}")
    print(f"      Mask mAP@0.5:0.95: {current_map:.4f}")
    
    # ============================================================
    # COLLECT METRICS FOR THIS EPOCH - ADD THIS SECTION
    # ============================================================
    # Read the results.csv from this epoch's training
    epoch_results_csv = f"/kaggle/working/mt_runs/mt_epoch_{epoch}/results.csv"
    
    if os.path.exists(epoch_results_csv):
        epoch_df = pd.read_csv(epoch_results_csv)
        epoch_df.columns = epoch_df.columns.str.strip()
        
        # Get the last row (final values for this epoch)
        last_row = epoch_df.iloc[-1]
        
        # Store metrics
        mt_metrics_history['epoch'].append(epoch + 1)
        mt_metrics_history['train/box_loss'].append(last_row.get('train/box_loss', 0))
        mt_metrics_history['train/seg_loss'].append(last_row.get('train/seg_loss', 0))
        mt_metrics_history['train/cls_loss'].append(last_row.get('train/cls_loss', 0))
        mt_metrics_history['train/dfl_loss'].append(last_row.get('train/dfl_loss', 0))
        mt_metrics_history['val/box_loss'].append(last_row.get('val/box_loss', 0))
        mt_metrics_history['val/seg_loss'].append(last_row.get('val/seg_loss', 0))
        mt_metrics_history['val/cls_loss'].append(last_row.get('val/cls_loss', 0))
        mt_metrics_history['val/dfl_loss'].append(last_row.get('val/dfl_loss', 0))
        mt_metrics_history['metrics/precision(B)'].append(last_row.get('metrics/precision(B)', 0))
        mt_metrics_history['metrics/recall(B)'].append(last_row.get('metrics/recall(B)', 0))
        mt_metrics_history['metrics/mAP50(B)'].append(last_row.get('metrics/mAP50(B)', 0))
        mt_metrics_history['metrics/mAP50-95(B)'].append(last_row.get('metrics/mAP50-95(B)', 0))
        mt_metrics_history['metrics/precision(M)'].append(last_row.get('metrics/precision(M)', 0))
        mt_metrics_history['metrics/recall(M)'].append(last_row.get('metrics/recall(M)', 0))
        mt_metrics_history['metrics/mAP50(M)'].append(last_row.get('metrics/mAP50(M)', 0))
        mt_metrics_history['metrics/mAP50-95(M)'].append(last_row.get('metrics/mAP50-95(M)', 0))
        
        print(f"   ✅ Metrics collected for epoch {epoch + 1}")
    
    # Track best model
    if current_map50 > best_map:
        best_map = current_map50
        best_epoch = epoch + 1
        student_model.save(best_model_path)
        print(f"\n   🎯 NEW BEST MODEL!")
        print(f"      Best mAP@0.5: {best_map:.4f}")
        print(f"      Saved to: {best_model_path}")
    else:
        print(f"\n   Current best: {best_map:.4f} (epoch {best_epoch})")

    
    
    # ============================================================
    # Step 6: Cleanup
    # ============================================================
    del student_model
    del teacher_model
    gc.collect()
    torch.cuda.empty_cache()
    
    print(f"\n✅ Epoch {epoch+1} complete!")

# Training summary
print("\n" + "="*70)
print("MEAN TEACHER TRAINING COMPLETE")
print("="*70)

print(f"\n🏆 Best Results:")
print(f"   Best mAP@0.5: {best_map:.4f}")
print(f"   Best epoch: {best_epoch}/{MEAN_TEACHER_EPOCHS}")
print(f"   Best model: {best_model_path}")


# ============================================================
# 👈 ADD THIS - CREATE COMBINED PLOT
# ============================================================
print("\n" + "="*70)
print("CREATING COMBINED TRAINING RESULTS PLOT")
print("="*70)

mt_results_df = pd.DataFrame(mt_metrics_history)
mt_results_df.to_csv('/kaggle/working/meanteacher_combined_results.csv', index=False)

fig, axes = plt.subplots(4, 4, figsize=(20, 20))
fig.suptitle('Mean Teacher SSL Training Results', fontsize=16, fontweight='bold')

# Row 1: Training losses
axes[0, 0].plot(mt_results_df['epoch'], mt_results_df['train/box_loss'], marker='o')
axes[0, 0].set_title('train/box_loss')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].plot(mt_results_df['epoch'], mt_results_df['train/seg_loss'], marker='o')
axes[0, 1].set_title('train/seg_loss')
axes[0, 1].grid(True, alpha=0.3)

axes[0, 2].plot(mt_results_df['epoch'], mt_results_df['train/cls_loss'], marker='o')
axes[0, 2].set_title('train/cls_loss')
axes[0, 2].grid(True, alpha=0.3)

axes[0, 3].plot(mt_results_df['epoch'], mt_results_df['train/dfl_loss'], marker='o')
axes[0, 3].set_title('train/dfl_loss')
axes[0, 3].grid(True, alpha=0.3)

# Row 2: Validation losses
axes[1, 0].plot(mt_results_df['epoch'], mt_results_df['val/box_loss'], marker='o')
axes[1, 0].set_title('val/box_loss')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].plot(mt_results_df['epoch'], mt_results_df['val/seg_loss'], marker='o')
axes[1, 1].set_title('val/seg_loss')
axes[1, 1].grid(True, alpha=0.3)

axes[1, 2].plot(mt_results_df['epoch'], mt_results_df['val/cls_loss'], marker='o')
axes[1, 2].set_title('val/cls_loss')
axes[1, 2].grid(True, alpha=0.3)

axes[1, 3].plot(mt_results_df['epoch'], mt_results_df['val/dfl_loss'], marker='o')
axes[1, 3].set_title('val/dfl_loss')
axes[1, 3].grid(True, alpha=0.3)

# Row 3: Box metrics
axes[2, 0].plot(mt_results_df['epoch'], mt_results_df['metrics/precision(B)'], marker='o')
axes[2, 0].set_title('metrics/precision(B)')
axes[2, 0].grid(True, alpha=0.3)

axes[2, 1].plot(mt_results_df['epoch'], mt_results_df['metrics/recall(B)'], marker='o')
axes[2, 1].set_title('metrics/recall(B)')
axes[2, 1].grid(True, alpha=0.3)

axes[2, 2].plot(mt_results_df['epoch'], mt_results_df['metrics/mAP50(B)'], marker='o')
axes[2, 2].set_title('metrics/mAP50(B)')
axes[2, 2].grid(True, alpha=0.3)

axes[2, 3].plot(mt_results_df['epoch'], mt_results_df['metrics/mAP50-95(B)'], marker='o')
axes[2, 3].set_title('metrics/mAP50-95(B)')
axes[2, 3].grid(True, alpha=0.3)

# Row 4: Mask metrics
axes[3, 0].plot(mt_results_df['epoch'], mt_results_df['metrics/precision(M)'], marker='o')
axes[3, 0].set_title('metrics/precision(M)')
axes[3, 0].grid(True, alpha=0.3)

axes[3, 1].plot(mt_results_df['epoch'], mt_results_df['metrics/recall(M)'], marker='o')
axes[3, 1].set_title('metrics/recall(M)')
axes[3, 1].grid(True, alpha=0.3)

axes[3, 2].plot(mt_results_df['epoch'], mt_results_df['metrics/mAP50(M)'], marker='o')
axes[3, 2].set_title('metrics/mAP50(M)')
axes[3, 2].grid(True, alpha=0.3)

axes[3, 3].plot(mt_results_df['epoch'], mt_results_df['metrics/mAP50-95(M)'], marker='o')
axes[3, 3].set_title('metrics/mAP50-95(M)')
axes[3, 3].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/kaggle/working/meanteacher_combined_results.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"\n✅ Combined results plot saved!")
# ============================================================



PHASE 2: MEAN TEACHER SSL TRAINING

📦 Available unlabeled images: 2727
   Using per epoch: 409
   Confidence threshold: 0.6

EPOCH 1/50

📦 Loading models...
   Using warmup weights
✅ Models loaded

🏷️  Generating pseudo-labels with teacher model...


Teacher (conf≥0.6): 100%|██████████| 409/409 [00:07<00:00, 55.38it/s]



   ✅ Generated 65 pseudo-labeled images

📦 Merging datasets...
   Labeled: 681
   Pseudo-labeled: 65
   Total: 746
✅ Datasets merged

🎓 Training student model...
   Training data: 746 images
   Epochs: 1 (incremental)
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/mt_merged.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.229      0.301      0.175      0.071      0.218      0.234      0.132     0.0429

1 epochs completed in 0.012 hours.
Optimizer stripped from /kaggle/working/mt_runs/mt_epoch_0/weights/last.pt, 23.8MB
Optimizer stripped from /kaggle/working/mt_runs/mt_epoch_0/weights/best.pt, 23.8MB

Validating /kaggle/working/mt_runs/mt_epoch_0/weights/best.pt...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 122/122 8.8it/s 13.8s0.1ss


...

                   all        974       1888      0.463      0.378      0.364      0.171      0.453      0.343      0.315       0.12

1 epochs completed in 0.012 hours.
Optimizer stripped from /kaggle/working/mt_runs/mt_epoch_49/weights/last.pt, 23.8MB
Optimizer stripped from /kaggle/working/mt_runs/mt_epoch_49/weights/best.pt, 23.8MB

Validating /kaggle/working/mt_runs/mt_epoch_49/weights/best.pt...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 122/122 9.1it/s 13.4s0.1ss


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888       0.46       0.38      0.364      0.171      0.453      0.343      0.315       0.12
Speed: 0.2ms preprocess, 5.3ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to /kaggle/working/mt_runs/mt_epoch_49
✅ Checkpoint saved: /kaggle/working/mt_epoch_49.pt

📊 Validating student model...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8s-seg summary (fused): 85 layers, 11,779,987 parameters, 0 gradients
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 27.7±4.0 MB/s, size: 35.2 KB)
val: Scanning /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/valid/labels... 974 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 974/974 606.9it/s 1.6s0.1s
WARNING ⚠️ val: Cache directory /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/valid is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95) 

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        974       1888      0.464      0.379      0.363      0.171      0.452      0.342      0.316      0.119
Speed: 1.1ms preprocess, 14.6ms inference, 0.0ms loss, 1.8ms postprocess per image
Results saved to /kaggle/working/runs/segment/val114

   📈 Metrics:
      Mask mAP@0.5:      0.3156
      Mask mAP@0.5:0.95: 0.1192

   🎯 NEW BEST MODEL!
      Best mAP@0.5: 0.3156
      Saved to: /kaggle/working/mt_best.pt

✅ Epoch 50 complete!

MEAN TEACHER TRAINING COMPLETE

🏆 Best Results:
   Best mAP@0.5: 0.3156
   Best epoch: 50/50
   Best model: /kaggle/working/mt_best.pt


In [16]:
# ============================================================
# CELL 9: FINAL TEST EVALUATION
# ============================================================
print("\n" + "="*70)
print("FINAL TEST EVALUATION")
print("="*70)

print(f"\n📊 Evaluating on test set...")
mt_final_model = YOLO(best_model_path)
mt_test_metrics = mt_final_model.val(split="test", verbose=False)

print(f"\n{'='*70}")
print(f"MEAN TEACHER - FINAL TEST RESULTS")
print(f"{'='*70}")
print(f"\n📊 Segmentation Metrics:")
print(f"   Mask mAP@0.5:      {mt_test_metrics.seg.map50:.4f}")
print(f"   Mask mAP@0.5:0.95: {mt_test_metrics.seg.map:.4f}")
print(f"\n📊 Detection Metrics:")
print(f"   Box mAP@0.5:       {mt_test_metrics.box.map50:.4f}")
print(f"   Box mAP@0.5:0.95:  {mt_test_metrics.box.map:.4f}")
print(f"{'='*70}")



FINAL TEST EVALUATION

📊 Evaluating on test set...
Ultralytics 8.3.234 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 1.2±1.1 ms, read: 32.7±7.2 MB/s, size: 34.6 KB)
val: Scanning /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/test/labels... 487 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 487/487 390.8it/s 1.2s0.0s
WARNING ⚠️ val: Cache directory /kaggle/input/car-damage-detection-3/Car-Damage detection.v1i.yolov8/test is not writable, cache not saved.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 31/31 2.6it/s 11.9s0.3s


/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        487        958      0.463      0.392      0.384      0.182      0.487      0.323       0.32      0.124
Speed: 1.4ms preprocess, 14.8ms inference, 0.0ms loss, 2.1ms postprocess per image
Results saved to /kaggle/working/runs/segment/val115

MEAN TEACHER - FINAL TEST RESULTS

📊 Segmentation Metrics:
   Mask mAP@0.5:      0.3202
   Mask mAP@0.5:0.95: 0.1239

📊 Detection Metrics:
   Box mAP@0.5:       0.3839
   Box mAP@0.5:0.95:  0.1821


In [17]:
# ============================================================
# CELL 10: SAVE BEST MODEL FOR DOWNLOAD
# ============================================================
print("\n" + "="*70)
print("SAVING MODELS")
print("="*70)

# Copy best model to root for easy download
shutil.copy(best_model_path, "/kaggle/working/meanteacher_best.pt")

print(f"\n✅ Models saved:")
print(f"   Best model: /kaggle/working/meanteacher_best.pt")
print(f"   ({os.path.getsize('/kaggle/working/meanteacher_best.pt')/1024/1024:.1f} MB)")


print("\n" + "="*70)
print("✅ MEAN TEACHER COMPLETE!")
print("="*70)
print(f"\n📥 Download 'meanteacher_best.pt' from Output panel")
print(f"   Size: {os.path.getsize('/kaggle/working/meanteacher_best.pt')/1024/1024:.1f} MB")
print(f"   Test mAP@0.5: {mt_test_metrics.seg.map50:.4f}")
print("\n" + "="*70)



SAVING MODELS

✅ Models saved:
   Best model: /kaggle/working/meanteacher_best.pt
   (22.6 MB)

✅ MEAN TEACHER COMPLETE!

📥 Download 'meanteacher_best.pt' from Output panel
   Size: 22.6 MB
   Test mAP@0.5: 0.3202

